In [ ]:
!pip install transformers

# embedding

1.lookup table 30522*768 token embedding

2.segment embedding 2*768

3.position embedding 512*768 max_length 512


layer norm

dropout

In [6]:
from transformers import BertModel, BertTokenizer
from transformers.models.bert import BertModel
import torch
import torch.nn as nn

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
test_sen = 'This is a test sentence'


In [9]:
input = tokenizer(test_sen,return_tensors = 'pt')

In [10]:
print(input)

{'input_ids': tensor([[ 101, 2023, 2003, 1037, 3231, 6251,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [11]:
input_ids = input['input_ids']
token_type_ids = input['token_type_ids']

In [12]:
input_ids

tensor([[ 101, 2023, 2003, 1037, 3231, 6251,  102]])

In [13]:
token_type_ids

tensor([[0, 0, 0, 0, 0, 0, 0]])

In [18]:
pos_ids = torch.arange(input_ids.shape[1])

In [19]:
pos_ids

tensor([0, 1, 2, 3, 4, 5, 6])

# word embedding

In [25]:
token_embedding = model.embeddings.word_embeddings(input_ids)

# segment embedding

In [15]:
segment_type_embedding = model.embeddings.token_type_embeddings(token_type_ids)

In [16]:
segment_type_embedding

tensor([[[ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         ...,
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086],
         [ 0.0004,  0.0110,  0.0037,  ..., -0.0066, -0.0034, -0.0086]]],
       grad_fn=<EmbeddingBackward0>)

In [23]:
segment_type_embedding.shape

torch.Size([1, 7, 768])

# position embedding

In [20]:
position_embedding = model.embeddings.position_embeddings(pos_ids)

In [21]:
position_embedding

tensor([[ 1.7505e-02, -2.5631e-02, -3.6642e-02,  ...,  3.3437e-05,
          6.8312e-04,  1.5441e-02],
        [ 7.7580e-03,  2.2613e-03, -1.9444e-02,  ...,  2.8910e-02,
          2.9753e-02, -5.3247e-03],
        [-1.1287e-02, -1.9644e-03, -1.1573e-02,  ...,  1.4908e-02,
          1.8741e-02, -7.3140e-03],
        ...,
        [-5.6087e-03, -1.0445e-02, -7.2288e-03,  ...,  2.0837e-02,
          3.5402e-03,  4.7708e-03],
        [-3.0871e-03, -1.8956e-02, -1.8930e-02,  ...,  7.4045e-03,
          2.0183e-02,  3.4077e-03],
        [ 6.4257e-03, -1.7664e-02, -2.2067e-02,  ...,  6.7531e-04,
          1.1108e-02,  3.7521e-03]], grad_fn=<EmbeddingBackward0>)

In [22]:
position_embedding.shape

torch.Size([7, 768])

# input embedding

In [26]:
input_embedding = token_embedding + position_embedding + segment_type_embedding

In [27]:
input_embedding

tensor([[[ 0.0316, -0.0411, -0.0564,  ...,  0.0021,  0.0044,  0.0219],
         [-0.0489,  0.0285, -0.0204,  ...,  0.0190,  0.0265, -0.0045],
         [-0.0469, -0.0156, -0.0336,  ...,  0.0117,  0.0135,  0.0109],
         ...,
         [ 0.0185, -0.0608, -0.0331,  ...,  0.0040, -0.0071, -0.1213],
         [ 0.0294, -0.0017, -0.0217,  ..., -0.0004, -0.0913, -0.0948],
         [-0.0077, -0.0166, -0.0123,  ..., -0.0310,  0.0124, -0.0064]]],
       grad_fn=<AddBackward0>)

In [28]:
embed = model.embeddings.LayerNorm(input_embedding)

In [30]:
embed = model.embeddings.dropout(embed)


In [29]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  